In [ ]:
from datetime import datetime, timedelta, timezone

import pandas as pd
import streamlit as st

from mainnet_launch.constants import AutopoolConstants, BASE_EUR


from mainnet_launch.database.schema.full import RebalanceEvents, AutopoolStates
from mainnet_launch.database.postgres_operations import (
    get_full_table_as_df_with_block,
    get_full_table_as_df_with_tx_hash,
)

rebalance_df = get_full_table_as_df_with_tx_hash(
    RebalanceEvents, where_clause=RebalanceEvents.autopool_vault_address == BASE_EUR.autopool_eth_addr
)
state_df = get_full_table_as_df_with_block(
    AutopoolStates, where_clause=AutopoolStates.autopool_vault_address == BASE_EUR.autopool_eth_addr
)

import pandas as pd
import plotly.express as px


def plot_turnover(resample_frequency: str, rebalance_df: pd.DataFrame, state_df: pd.DataFrame):
    # Group both by the same frequency (e.g., "W-MON" for weeks starting Monday)
    end_of_period_safe_value_out = rebalance_df.groupby(pd.Grouper(freq=resample_frequency))["safe_value_out"].sum()
    end_of_period_nav = state_df.groupby(pd.Grouper(freq=resample_frequency))["total_nav"].mean()

    # Align by index
    df = pd.concat([end_of_period_safe_value_out, end_of_period_nav], axis=1).round(2)
    df["turnover"] = df["safe_value_out"] / df["total_nav"]

    return df


frequencies = ["W-SUN", "ME"]

for freq in frequencies:
    st.write(f"## Resample Frequency: {freq}")
    df = plot_turnover(freq, rebalance_df, state_df)
    fig = px.bar(df, x=df.index, y="turnover", hover_data=["safe_value_out", "total_nav"], title="Turnover Rate")
    fig.show()
    px.bar(df, x=df.index, y="turnover", hover_data=["safe_value_out", "total_nav"], title="Turnover Rate")

In [17]:
state_df.resample("7d")["total_nav"].last()

datetime
2025-08-05 00:00:00+00:00    1.304470e+06
2025-08-12 00:00:00+00:00    1.612020e+06
2025-08-19 00:00:00+00:00    1.808985e+06
2025-08-26 00:00:00+00:00    1.903575e+06
2025-09-02 00:00:00+00:00    1.891866e+06
2025-09-09 00:00:00+00:00    1.794477e+06
2025-09-16 00:00:00+00:00    2.082862e+06
Freq: 7D, Name: total_nav, dtype: float64

In [19]:
rebalance_df.resample("7d")["safe_value_out"].sum()

datetime
2025-08-06 00:00:00+00:00    1.974239e+06
2025-08-13 00:00:00+00:00    2.336136e+06
2025-08-20 00:00:00+00:00    2.002607e+06
2025-08-27 00:00:00+00:00    3.762346e+05
2025-09-03 00:00:00+00:00    1.842082e+06
2025-09-10 00:00:00+00:00    2.580172e+05
2025-09-17 00:00:00+00:00    2.893609e+05
Freq: 7D, Name: safe_value_out, dtype: float64

2025-09-18 12:20:42.335 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 12:20:42.336 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 12:20:42.336 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 12:20:42.337 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


2025-09-18 12:20:42.371 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 12:20:42.372 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 12:20:42.372 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 12:20:42.372 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
# def fetch_and_render_turnover_data(autopool: AutopoolConstants):
#     turnover_summary = fetch_turnover_data(autopool)
#     st.header(f"{autopool.name} Turnover")
#     st.table(turnover_summary)


# def fetch_turnover_data(autopool: AutopoolConstants) -> pd.DataFrame:
#     clean_rebalance_df = None  # fetch_rebalance_events_df(autopool)

#     def _is_a_rebalance_between_the_same_destination(row) -> bool:
#         # moveName = f"{out_destination_symbol} -> {in_destination_symbol}"
#         destination_symbols = row["moveName"].split(" -> ")
#         if len(destination_symbols) == 2:
#             out_destination_symbol, in_destination_symbol = destination_symbols
#             if out_destination_symbol == in_destination_symbol:
#                 return True
#         return False

#     clean_rebalance_df["is_rebalance_between_the_same_destination"] = clean_rebalance_df.apply(
#         _is_a_rebalance_between_the_same_destination, axis=1
#     )

#     pricePerShare_df = fetch_destination_summary_stats(autopool, "pricePerShare")
#     ownedShares_df = fetch_destination_summary_stats(autopool, "ownedShares")
#     allocation_df = pricePerShare_df * ownedShares_df
#     total_nav_series = allocation_df.sum(axis=1)

#     today = datetime.now(timezone.utc)

#     seven_days_ago = today - timedelta(days=7)
#     thirty_days_ago = today - timedelta(days=30)
#     one_year_ago = today - timedelta(days=365)

#     records = []
#     for window_name, window in zip(
#         ["seven_days_ago", "thirty_days_ago", "one_year_ago"], [seven_days_ago, thirty_days_ago, one_year_ago]
#     ):

#         recent_df = clean_rebalance_df[clean_rebalance_df.index >= window]
#         rebalance_count = len(recent_df)

#         avg_tvl = float(total_nav_series[total_nav_series.index >= window].mean())

#         total_volume_with_rebalances_to_same_destination = recent_df["outEthValue"].sum()
#         total_volume_without_rebalances_to_same_destination = recent_df[
#             ~recent_df["is_rebalance_between_the_same_destination"]
#         ]["outEthValue"].sum()

#         record = {
#             "window": window_name,
#             "rebalances": rebalance_count,
#             "avg_tvl": round(avg_tvl, 2),
#             "volume_with_rebalances_to_self": round(total_volume_with_rebalances_to_same_destination, 2),
#             "volume_without_rebalances_to_self": round(total_volume_without_rebalances_to_same_destination, 2),
#             "turnover_with_rebalances_to_self": round(total_volume_with_rebalances_to_same_destination / avg_tvl, 3),
#             "turnover_without_rebalances_to_self": round(
#                 total_volume_without_rebalances_to_same_destination / avg_tvl, 3
#             ),
#         }
#         records.append(record)

#     turnover_summary = pd.DataFrame.from_records(records)
#     return turnover_summary


# if __name__ == "__main__":
#     fetch_and_render_turnover_data(BASE_ETH)

#     # for a in ALL_AUTOPOOLS:
#     #     fetch_and_render_turnover_data(a)